# Data Scraping [Twitter & Reddit]

https://www.kaggle.com/code/nikhilkhetan/utility-data-scraping-twitter-

### Table of Contents

- Introduction to Data Scraping
- Data Scraping from Twitter
- Data Scraping from Reddit

#### Introduction to Data Scraping¶

Data is available everywhere. To perform various Data Science experiments we often need to extract data from various sources.

We will use the codes in this notebook to extract data from some of the popular websites (Twitter and Reddit). The codes published here can be used to extract tweets based on the user's requirement and converted to a Pandas dataset. For Reddit, you can scrape an entire subreddit and convert it into a Pandas dataset.

Method 3 for Twitter uses snscrape which requires Python version 3.8. As Kaggle notebooks run on Python version 3.7 and I could not find a reliable work around on this, the codes for Method 3 have been commented to avoid error on execution.

Please use Python version 3.8 or higher on Jupyter notebooks to run the codes for Twitter Method 3.

##### Data Scraping from Twitter

You need a Twitter Developer account for Method 1 and 2

Method 1 - Twitter Scraper using Keywords

Extract all tweets based on a keyword, e.g. Covid-19, DataScience, etc.

In [ ]:
!pip install tweepy

In [ ]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

In [ ]:
consumer_api_key = "nzvUSC1mrZwM07vEf0mENua1G"
consumer_api_secret = "cRsQgCynk9zF4yuKciK3x09BebKYysoLIicDLgsOpicSiDdtLP"

auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

##### Query Tweets

In [ ]:
search_words = "#datascience -filter:retweets"
date_since = "2021-01-01"

# Collect tweets
tweets = tw.Cursor(api.search_tweets,
              q=search_words,
              lang="en").items(10)
              # since=date_since).items(500)

In [ ]:
# Recuperando los tweets
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

In [ ]:
print(f"new tweets retrieved: {len(tweets_copy)}")

Populate the Dataset. Extract the information contained in a tweet into a Pandas dataframe

In [ ]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

In [ ]:
tweets_df.head()

Save the Data. Read past data

Skip this part for the very first execution as there is no past data. Instead save your dataframe directly to a csv and use this part for the next runs

In [ ]:
tweets_old_df = pd.read_csv("../input/data-scraping-data-science-tweets/datascience_tweets.csv")

print(f"past tweets: {tweets_old_df.shape}")

Merge Past and Present Data¶

In [ ]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)

print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

Drop Duplicates

In [ ]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

Export the updated data

In [ ]:
tweets_all_df.to_csv("datascience_tweets.csv", index=False)

##### Method 2 - Tweet Extractor using Twitter username¶

Extract tweets of a particular user using the screen_name

In [20]:
import tweepy
import pandas as pd

In [25]:
access_key = "89910029-ISYDmGP700BFyiz6FxwB7k08iBGMqmzQNF8rS6yb8"
access_secret = "bLfU8tUoGUJ3oAHggqErO1TltVP0Vt57BJ42M6P0gSVrQ"
consumer_api_key = "nzvUSC1mrZwM07vEf0mENua1G"
consumer_api_secret = "cRsQgCynk9zF4yuKciK3x09BebKYysoLIicDLgsOpicSiDdtLP"

auth = tweepy.OAuthHandler(consumer_api_key, consumer_api_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

Función recuperar tweets

In [26]:
def get_all_tweets(screen_name):
    alltweets = []
    new_tweets = api.user_timeline(screen_name = screen_name,count=200) #Using the Twitter user_timeline API
    alltweets.extend(new_tweets)
    oldest = alltweets[-1].id - 1
    
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        alltweets.extend(new_tweets)
        oldest = alltweets[-1].id - 1
        print ("...%s tweets downloaded so far" % (len(alltweets)))

        data=[[obj.user.screen_name,obj.user.name,obj.user.id_str,obj.user.description.encode("utf8"),obj.created_at.year,obj.created_at.month,obj.created_at.day,"%s.%s"%(obj.created_at.hour,obj.created_at.minute),obj.id_str,obj.text.encode("utf8")] for obj in alltweets ]
        dataframe=pd.DataFrame(data,columns=['screen_name','name','twitter_id','description','year','month','date','time','tweet_id','tweet'])
        dataframe.to_csv("%s_tweets.csv"%(screen_name),index=False)

Pasamos el usuario de la cuenta que queremos recuperar tweets

In [27]:
if __name__ == '__main__':
	get_all_tweets("jack")

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

##### Method 3 - Tweet Extractor using snscrape

No Authentication required for using snscrape

In [28]:
import os
!pip install --upgrade git+https://github.com/JustAnotherArchivist/snscrape@master

  Cloning https://github.com/JustAnotherArchivist/snscrape (to revision master) to c:\users\javic\appdata\local\temp\pip-req-build-r_htisxp
  Resolved https://github.com/JustAnotherArchivist/snscrape to commit 614d4c2029a62d348ca56598f87c425966aaec66
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for snscrape: filename=snscrape-0.7.0.20230622-py3-none-any.whl size=75348 sha256=dd4588e9a8f0844129a91f264b4edec233e709b8df92b6ef9d4f18df21d06e1d
  Stored in directory: C:\Users\javic\AppData\Local\Temp\pip-ephem-wheel-cache-sh_bsf4a\wheels\46\8a\5a\f5cb4564c6e25c14215a8978ab35482164450daf8c35b3e91a
Successfully built snscrape


  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape 'C:\Users\javic\AppData\Local\Temp\pip-req-build-r_htisxp'

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
# Pass in the username whose tweets you want to pull
os.system("snscrape --jsonl twitter-search 'from:JeffBezos'> user-tweets.json")

1

In [31]:
import pandas as pd

# Reads the json generated from the CLI commands above and creates a pandas dataframe
tweets_df = pd.read_json('user-tweets.json', lines=True)

tweets_df.shape

(0, 0)

In [32]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


Mas ejemplos

In [33]:
os.system("snscrape --jsonl --max-results 100 twitter-search 'from:user'> user-tweets.json")

os.system("snscrape --jsonl --max-results 500 --since 2020-06-01 twitter-search 'its the elephant until:2020-07-31' > text-query-tweets.json")

2

Creating a dataframe from the results of the examples above

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
tweets_list1 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:user').get_items()):
    if i>100:
        break
    tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username])
    
# Creating a dataframe from the tweets list above 
tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
tweets_df1

Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22from%3Auser%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_everywhere_api_en

ScraperException: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22from%3Auser%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3Atrue%2C%22view_counts_ev

In [ ]:
import pandas as pd

# Creating list to append tweet data to
tweets_list2 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('its the elephant since:2020-06-01 until:2020-07-31').get_items()):
    if i>500:
        break
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username])
   
# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
tweets_df2

Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22its%20the%20elephant%20since%3A2020-06-01%20until%3A2020-07-31%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_translatable_e

ScraperException: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22its%20the%20elephant%20since%3A2020-06-01%20until%3A2020-07-31%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is

##### Data Scraping from Reddit

In [37]:
pip install praw

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import os
import praw
import pandas as pd
import datetime as dt
from tqdm import tqdm
import time

In [39]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [40]:
#fill in the below Authentication details from Reddit
def reddit_connection():
    personal_use_script = user_secrets.get_secret("PERSONAL_USE_SCRIPT")
    client_secret = user_secrets.get_secret("CLIENT_SECRET")
    user_agent = user_secrets.get_secret("USER_AGENT")
    username = user_secrets.get_secret("USERNAME")
    password = user_secrets.get_secret("PASSWORD")

    reddit = praw.Reddit(client_id=personal_use_script, \
                         client_secret=client_secret, \
                         user_agent=user_agent, \
                         username=username, \
                         password='')
    return reddit

In [41]:
def build_dataset(reddit, search_words='gameofthrones', items_limit=None):

    # Collect reddit posts
    subreddit = reddit.subreddit(search_words)
    new_subreddit = subreddit.new(limit=items_limit)
    topics_dict = { "title":[],
                "score":[],
                "id":[], "url":[],
                "comms_num": [],
                "created": [],
                "body":[]}

    print(f"retreive new reddit posts ...")
    for submission in tqdm(new_subreddit):
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)

    for comment in tqdm(subreddit.comments(limit=None)):
        topics_dict["title"].append("Comment")
        topics_dict["score"].append(comment.score)
        topics_dict["id"].append(comment.id)
        topics_dict["url"].append("")
        topics_dict["comms_num"].append(0)
        topics_dict["created"].append(comment.created)
        topics_dict["body"].append(comment.body)

    topics_df = pd.DataFrame(topics_dict)
    print(f"new reddit posts retrieved: {len(topics_df)}")
    topics_df['timestamp'] = topics_df['created'].apply(lambda x: get_date(x))

    return topics_df

In [42]:
def update_and_save_dataset(topics_df):   
    file_path = "reddit_GoT.csv"
    if os.path.exists(file_path):
        topics_old_df = pd.read_csv(file_path)
        print(f"past reddit posts: {topics_old_df.shape}")
        topics_all_df = pd.concat([topics_old_df, topics_df], axis=0)
        print(f"new reddit posts: {topics_df.shape[0]} past posts: {topics_old_df.shape[0]} all posts: {topics_all_df.shape[0]}")
        topics_new_df = topics_all_df.drop_duplicates(subset = ["id"], keep='last', inplace=False)
        print(f"all reddit posts: {topics_new_df.shape}")
        topics_new_df.to_csv(file_path, index=False)
    else:
        print(f"reddit posts: {topics_df.shape}")
        topics_df.to_csv(file_path, index=False)

In [ ]:
if __name__ == "__main__": 
	reddit = reddit_connection()
	topics_data_df = build_dataset(reddit)
	update_and_save_dataset(topics_data_df)

Tratamiento datos recuperados